In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import joblib
import torch.nn as nn
import numpy as np
import cv2
import torch.nn.functional as F
import time
import cnn_models
from rnn_model import WordLSTM
from autocorrect import Speller
from gensim.corpora import Dictionary
import matplotlib.pyplot as plt
import time
import predict_next_word
from predict_next_word import predict
from predict_next_word import sample
%matplotlib inline

Loading label binarizer...


In [4]:
# load label binarizer
pred_text = ""
lb = joblib.load('../outputs/lb.pkl')
model = cnn_models.CustomCNN()
model.load_state_dict(torch.load('../outputs/model.pth', map_location=torch.device('cpu')))
print(model)
rnn_model = WordLSTM()
rnn_model.load_state_dict(torch.load('../outputs/word_predictor.pth', map_location=torch.device('cpu')))
print(sample(rnn_model,'i'))
print(rnn_model)
print('Model loaded')

def hand_area(img):
    hand = img[100:324, 100:324]
    hand = cv2.resize(hand, (224,224))
    return hand

cap = cv2.VideoCapture(0)
if (cap.isOpened() == False):
    print('Error while trying to open camera. Plese check again...')
# get the frame width and height
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
# define codec and create VideoWriter object
out = cv2.VideoWriter('../outputs/asl_'+str(time.time())+'.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width,frame_height))

# 'c' to store registered letter from previous frame
c = ''
cur_pred_word = ''
# Initialize counter to only append registered letter if counter = 30
counter = 0
spell = Speller()
next_word=''
# read until end of video
while(cap.isOpened()):
    # capture each frame of the video
    ret, frame = cap.read()
    # get the hand area on the video capture screen
    cv2.rectangle(frame, (100, 100), (324, 324), (20, 34, 255), 2)
    hand = hand_area(frame)
    image = hand
    
    image = np.transpose(image, (2, 0, 1)).astype(np.float32)
    image = torch.tensor(image, dtype=torch.float)
    image = image.unsqueeze(0)
    
    outputs = model(image)
    _, preds = torch.max(outputs.data, 1)
    cv2.putText(frame, lb.classes_[preds], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
#     pred_text = pred_text +" "+ lb.classes_[preds]
    
    # Show what is the current string of letters
    cv2.putText(frame, "Current String:" + pred_text, (10, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    if c == lb.classes_[preds]:
        c = lb.classes_[preds]
        counter += 1
        # Counter for how long a letter must remain in box before registering
        if counter == 60:
            cv2.putText(frame, "Letter added: " + lb.classes_[preds], (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
            if lb.classes_[preds] == "space":
                if len(pred_text)!=0:
                    next_word=sample(rnn_model,pred_text.lower())
                pred_text = pred_text + " "
            else:
                pred_text = pred_text + lb.classes_[preds]
            print(lb.classes_[preds] + " Added into string")
            
            # only predict word when letter is added
            cur_pred_word = spell(pred_text)
            counter = 0

    cv2.putText(frame, "Next possible word:" + next_word, (10, 460), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    
    if c != lb.classes_[preds]:
        c = lb.classes_[preds]
        counter = 0
    
    # Display current predicted word
    cv2.putText(frame, "Current Pred Word: " + cur_pred_word, (10, 430), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.putText(frame, str(counter) + "/60", (150, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
    cv2.imshow('image', frame)
    out.write(frame)
    # press `q` to exit
    if cv2.waitKey(27) & 0xFF == ord('q'):
        # Clear cur_pred_word
        cur_pred_word = ''
        break
# release VideoCapture()
cap.release()
# close all frames and video windows
cv2.destroyAllWindows()

print("\n\n\nPredicted Output:-")
print(pred_text)


CustomCNN(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=29, bias=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
Not found in our dictionary. Try another sign language word sequence.
WordLSTM(
  (emb_layer): Embedding(16120, 200)
  (lstm): LSTM(200, 256, num_layers=4, batch_first=True, dropout=0.3)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=16120, bias=True)
)
Model loaded
D Added into string
nothing Added into string
nothing Added into string



Predicted Output:-
Dnothingnothing


In [5]:
t = str(pred_text)
t = t.replace("nothing", "")
t = t.replace("del", "")
# t = t.replace("space", "")
t

''